In [18]:
from dotenv import load_dotenv
load_dotenv()

# .env 파일 강제 로드
load_dotenv(override=True)

True

In [19]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader('C:\Workspaces\AI-Pharmacist\KimHaebin\drug_info.csv', encoding='utf-8')
drug_info_document = loader.load()
print(len(drug_info_document))
print(drug_info_document[0])

4756
page_content='﻿업체명: 동화약품(주)
제품명: 활명수
품목기준코드: 195700020
효능: 이 약은 식욕감퇴(식욕부진), 위부팽만감, 소화불량, 과식, 체함, 구역, 구토에 사용합니다.
복용방법: 만 15세 이상 및 성인은 1회 1병(75 mL), 만 11세이상~만 15세미만은 1회 2/3병(50 mL), 만 8세 이상~만 11세 미만은 1회 1/2병(37.5 mL), 만 5세 이상~만 8세 미만은 1회 1/3병(25 mL), 만 3세 이상~만 5세 미만은 1회 1/4병(18.75 mL), 만 1세 이상~만 3세 미만은 1회 1/5병(15 mL), 1일 3회 식후에 복용합니다. 복용간격은 4시간 이상으로 합니다.
알아야할 점: 
사용상 주의사항: 만 3개월 미만의 젖먹이는 이 약을 복용하지 마십시오.

이 약을 복용하기 전에 만 1세 미만의 젖먹이, 임부 또는 임신하고 있을 가능성이 있는 여성, 카라멜에 과민증 환자 또는 경험자, 나트륨 제한 식이를 하는 사람은 의사 또는 약사와 상의하십시오.

정해진 용법과 용량을 잘 지키십시오.

어린이에게 투여할 경우 보호자의 지도 감독하에 투여하십시오.

1개월 정도 복용하여도 증상의 개선이 없을 경우 복용을 즉각 중지하고 의사 또는 약사와 상의하십시오.
병용금지 약/음식: 
이상반응 가능성: 
보관방법: 습기와 빛을 피해 실온에서 보관하십시오.

어린이의 손이 닿지 않는 곳에 보관하십시오.' metadata={'source': 'C:\\Workspaces\\AI-Pharmacist\\KimHaebin\\drug_info.csv', 'row': 0}


In [20]:
import os
PINECONE_INDEX_NAME_2 = os.getenv("PINECONE_INDEX_NAME_2")
PINECONE_NAMESPACE_2 = os.getenv("PINECONE_NAMESPACE_2")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

print(PINECONE_INDEX_NAME_2)
print(PINECONE_NAMESPACE_2)
print(PINECONE_API_KEY)

ai-drug-info
ai-drug-info-ns
pcsk_5TkRik_EzGTnXtpN1c26rjgbrhySAUqgAAHhR9QEmje4vwZkMoEKiADwUKrPjkEAQwfsTs


In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [21]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os

PINECONE_INDEX_NAME_2 = os.getenv("PINECONE_INDEX_NAME_2")
PINECONE_NAMESPACE_2 = os.getenv("PINECONE_NAMESPACE_2")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# 문서 배치 크기 설정 (예: 1000개씩 나눔)
batch_size = 500
total_documents = len(drug_info_document)
# # 문서에 대한 임베딩 생성
# document_embeddings = embeddings.embed_documents([doc.page_content for doc in preg_dur_document])

# 문서를 배치로 나누어 처리
for i in range(0, total_documents, batch_size):
    batch = drug_info_document[i:i+batch_size]
    # embeddings_batch = document_embeddings[i:i+batch_size]  # 배치에 맞는 임베딩
    
    # Pinecone에 배치 업로드
    vector_db = PineconeVectorStore.from_documents(
        batch,
        embedding=embeddings,
        index_name=PINECONE_INDEX_NAME_2,
        namespace=PINECONE_NAMESPACE_2,
        pinecone_api_key=PINECONE_API_KEY
    )

    # 배치 업로드 완료 메시지
    print(f"Batch {i // batch_size + 1} uploaded, {min(i + batch_size, total_documents)} documents processed.")

Batch 1 uploaded, 500 documents processed.
Batch 2 uploaded, 1000 documents processed.
Batch 3 uploaded, 1500 documents processed.
Batch 4 uploaded, 2000 documents processed.
Batch 5 uploaded, 2500 documents processed.
Batch 6 uploaded, 3000 documents processed.
Batch 7 uploaded, 3500 documents processed.
Batch 8 uploaded, 4000 documents processed.
Batch 9 uploaded, 4500 documents processed.
Batch 10 uploaded, 4756 documents processed.


In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', """
Persona: 당신은 전문적인 약사입니다. 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등을 알고 있으며 일반인을 대상으로 설명이 가능합니다.

Role: 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등에 context를 기반으로 약품에 대해 아무것도 모르는 일반인이 이해하도록 간략하게 설명해줍니다.

"""),
    ('user', '''
    query 질문에 대해 context만 참조해서 알려줘.
    
    question: {query}  
    context: {context}
    ''')
])
prompt

ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nPersona: 당신은 전문적인 약사입니다. 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등을 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등에 context를 기반으로 약품에 대해 아무것도 모르는 일반인이 이해하도록 간략하게 설명해줍니다.\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, template='\n    query 질문에 대해 context만 참조해서 알려줘.\n    \n    question: {query}  \n    context: {context}\n    '), additional_kwargs={})])

In [24]:
query = "에이서캡슐(아세클로페낙)"
retriever = vector_db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 10}
)
retrievals = retriever.batch([query])
retrievals

[[Document(id='410a064d-1954-49ea-9db7-464a8087e311', metadata={'row': 2526.0, 'source': 'C:\\Workspaces\\AI-Pharmacist\\KimHaebin\\drug_info.csv'}, page_content='\ufeff업체명: (주)한국글로벌제약\n제품명: 뮤코틸캡슐(아세틸시스테인)\n품목기준코드: 201404707\n효능: 이 약은 급ㆍ만성기관지염, 기관지천식, 후두염, 부비동염, 낭성섬유증의 질환에서\xa0객담배출곤란에 사용합니다.\n복용방법: 급성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 3회, 6~14세는 1회 1캡슐(200 mg)씩, 1일 2회 식전에 소량의 물과 함께 복용합니다.만성질환에 성인은 1회 1캡슐(200 mg)씩, 1일 2회, 6~14세는 1회\xa0\xa01/2\xa0캡슐(100 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.낭성섬유증에 6세 이상은 1회 1캡슐(200 mg)씩, 1일 3회 식전에 소량의 물과 함께 복용합니다.\n알아야할 점: \n사용상 주의사항: 이 약에 과민증 환자,\xa0위•십이지장궤양,\xa02세 미만 영아, 갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등의 유전적인 문제가 있는 환자는 이 약을 복용하지 마십시오.이 약을 복용하기 전에\xa0임부 또는 임신하고 있을 가능성이 있는 여성 및 수유부는 의사 또는 약사와 상담하십시오.\n병용금지 약/음식: 경구용 항생제(아목시실린, 세푸록심, 독시사이클린, 에리트로마이신, 티암페니콜)와\xa0함께 복용 시 적어도 2시간\xa0간격을 두고 복용하십시오.\n이상반응 가능성: 드물게 구역, 구토, 구내염, 가슴쓰림, 설사, 두드러기, 가려움, 발진, 두통, 이명, 기관지경련(특히 천식 환자), 빈맥, 고혈압 등이 나타나는 경우\xa0즉각 복용을 중지하고\xa0의사 또는 약사와 상담하십시오.\n보관방법: 실온에서 보관하십시오.어린이의 손이 닿

In [15]:
prompt.invoke({'query': query, 'context': retriever})

ChatPromptValue(messages=[SystemMessage(content='\nPersona: 당신은 전문적인 약사입니다. 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등을 알고 있으며 일반인을 대상으로 설명이 가능합니다.\n\nRole: 약의 효능, 복용방법, 알아야할 점, 사용상 주의사항, 병용금지 약/음식, 이상반응 가능성, 보관방법 등에 context를 기반으로 약품에 대해 아무것도 모르는 일반인이 이해하도록 간략하게 설명해줍니다.\n\n', additional_kwargs={}, response_metadata={}), HumanMessage(content="\n    query 질문에 대해 context만 참조해서 알려줘.\n    \n    question: 소화사 안돼서 활명수 먹으려해. 주의할 게 있을까?  \n    context: tags=['PineconeVectorStore', 'OpenAIEmbeddings'] vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x00000188A7065410> search_kwargs={'k': 10}\n    ", additional_kwargs={}, response_metadata={})])

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o',
    temperature=0.5
)

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RunnablePassThrough: invoke()에 전달된 인자를 받아서 그대로 출력
chain = {'query': RunnablePassthrough(), 'context': retriever} | prompt | model | StrOutputParser() 

chain.invoke(query)

'활명수를 복용하실 때 주의할 점에 대해 설명드리겠습니다.\n\n1. **복용 방법**: 활명수는 일반적으로 소화불량, 식욕감퇴, 과식 등으로 인한 소화 문제를 완화하는 데 사용됩니다. 제품에 따라 복용 방법이 다를 수 있으니, 포장지에 있는 지시사항을 잘 따라주세요.\n\n2. **사용상 주의사항**: 활명수는 일반적으로 안전하게 사용될 수 있지만, 알레르기 체질이거나 특정 성분에 과민 반응이 있는 경우 주의가 필요합니다. 특히, 만 7세 이하의 어린이는 복용하지 않는 것이 좋습니다.\n\n3. **병용금지 약/음식**: 특정 약물이나 음식과 함께 복용할 때 주의가 필요할 수 있습니다. 다른 약물을 복용 중이시라면, 약사나 의사와 상담하시는 것이 좋습니다.\n\n4. **이상반응 가능성**: 드물게 복부 팽만감, 구역, 변비 또는 설사 등의 이상반응이 나타날 수 있습니다. 이러한 증상이 나타나면 복용을 중단하고 의사나 약사와 상담하세요.\n\n5. **보관방법**: 습기와 빛을 피하여 실온에서 보관하시고, 어린이의 손이 닿지 않는 곳에 보관하십시오.\n\n활명수를 복용하시기 전에 위의 사항들을 확인하시고, 필요시 전문가와 상담하시는 것이 중요합니다.'